In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
224,DECEMBER,25,One Night in Miami,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,[217]
225,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,[218]
226,DECEMBER,25,Sylvie's Love,Amazon Studios,Eugene Ashe (director/screenplay); Tessa Thomp...,[219]
227,DECEMBER,30,Monster Hunter,Screen Gems / Constantin Film / Tencent Pictur...,Paul W. S. Anderson (director/screenplay); Mil...,[220]


In [10]:
df_2020 = df[['Title','Cast and crew']]

In [11]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
224,One Night in Miami,Regina King (director); Kemp Powers (screenpla...
225,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...
226,Sylvie's Love,Eugene Ashe (director/screenplay); Tessa Thomp...
227,Monster Hunter,Paul W. S. Anderson (director/screenplay); Mil...


In [12]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '840ffda78b661c109c8cf357bf69219b'

In [18]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [19]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

IndexError: list index out of range

In [20]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
224,One Night in Miami,Regina King (director); Kemp Powers (screenpla...
225,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...
226,Sylvie's Love,Eugene Ashe (director/screenplay); Tessa Thomp...
227,Monster Hunter,Paul W. S. Anderson (director/screenplay); Mil...


In [21]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [22]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-22-bc52fe9b0c35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [23]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [24]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-24-42c5f763f942>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [25]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [26]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-26-ca9c918a7840>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [27]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [28]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [29]:
df_2020

,Title,Cast and crew,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
...,...,...,...,...,...,...
224,One Night in Miami,Regina King (director); Kemp Powers (screenpla...,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
225,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie
226,Sylvie's Love,Eugene Ashe (director/screenplay); Tessa Thomp...,Eugene Ashe,Tessa Thompson,Nnamdi Asomugha,Ryan Michelle Bathe
227,Monster Hunter,Paul W. S. Anderson (director/screenplay); Mil...,Paul W. S. Anderson,Milla Jovovich,Tony Jaa,"Tip ""T.I."" Harris"


In [28]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [29]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [30]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,Inherit the Viper
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,The Sonata
...,...,...,...,...,...,...
150,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,Top Gun: Maverick
151,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,The Croods 2
152,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,Respect
153,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,The Last Duel


In [32]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [33]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      2
actor_3_name     11
genres            3
movie_title       1
comb             14
dtype: int64

In [34]:
new_df20 = new_df20.dropna(how='any')

In [35]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [36]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

<ipython-input-36-83e83e91073b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [37]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,the sonata,Freya Tingley Simon Abkarian Rutger Hauer Andr...
...,...,...,...,...,...,...,...
150,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Jose...
151,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
152,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,respect,Jennifer Hudson Forest Whitaker Marlon Wayans ...
153,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scot...


In [38]:
old_df = pd.read_csv('final_data.csv')

In [39]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [40]:
final_df = old_df.append(new_df20,ignore_index=True)

In [41]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6005,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Jose...
6006,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
6007,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,respect,Jennifer Hudson Forest Whitaker Marlon Wayans ...
6008,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scot...


In [42]:
final_df.to_csv('main_data.csv',index=False)